## Dataset Description

You can train Talking Head Anime with two different type of datasets:

1. Images Dataset (recommended)
2. 3D-models Dataset

If you don't have appropriate dataset, you may want to follow instructions below or write your own dataset code with your data.

If you have any, or have own code for your dataset, you may skip these.

### Images dataset:

* set of images consist of (model_base, model_shaped, model_shaped_rotated) pairs.
* Detailed description of pairs:
    * model_base: Image of model with rest(base) pose. `base.png`
    * model_shaped: Image of model with its face parts' shape changed. `shape_0.57_0.0_0.5.png`
    * model_shaped_rotated: Image of model of `model_shaped`'s head rotated with XYZ
      axis.`pose_0.57_0.0_0.5_0.41_-0.68_1.png`

<table>
<tr>
    <th> Type </th>
    <th width=256>Model base</th>
    <th width=256>Model shaped</th>
    <th width=256>Model shaped rotated</th> 
</tr>

<tr>
    <th> Description </th>
    <td> Rendered Image of a 3d model with rest pose </td>
    <td> Rendered Image of a 3d model with its face expression changed. <br> Mouth: 0.57 open, Left eye: 0.0 closed, Right eye: 0.5 closed. </td>
    <td> Rendered Image of a 3d model with its face expression changed, and then head rotated. <br> Mouth: 0.57 open, Left eye: 0.0 closed, Right eye: 0.5 closed. <br> Head rotated  with x axis: 0.41 pi/2, y_axis: -0.68 pi/2, z_axis: 1 pi/2 </td>
</tr>
    
<tr>
    <th> Image </th>
    <td><img src="src/images/base.png?v=1" width=256/></td> 
    <td><img src="src/images/shape_0.57_0.0_0.5.png" width=256/></td> 
    <td><img src="src/images/pose_0.57_0.0_0.5_0.41_-0.68_1.png" width=256/></td>
</tr>
</table>



### 3D-models dataset:

Note that 3D-models dataset may suffer with extermely low speed when training.

* Set of models where each files' extension is one of `.pmx`, `.pmd` or `.vrm`.

Example:
```
[dhchoi@localhost 3d_models]$ cat all_blends.txt | head -3
/root/talking_head_anime_2/data/3d_models/blends/3d.nicovideo__10003__こんにゃく式戌亥とこver1.0/こんにゃく式戌亥とこver1.0/戌亥とこ.pmx
/root/talking_head_anime_2/data/3d_models/blends/3d.nicovideo__10024__(にじさんじ)ニュイ・ソシエールVer1.00/(にじさんじ)ニュイ・ソシエールVer1.00/models/ニュイ・ソシエール(帽子無し).pmx
/root/talking_head_anime_2/data/3d_models/blends/3d.nicovideo__10024__(にじさんじ)ニュイ・ソシエールVer1.00/(にじさんじ)ニュイ・ソシエールVer1.00/models/ニュイ・ソシエール.pmx
```

----

## Downloading 3D Models

Here, we'll download 3D models and filter usable models for Talking Head Anime.

### Downloading 3D models

I batch-crawled 3d models from [3d.nicovideo.jp](3d.nicovideo.jp) and [bowlroll.net](bowlroll.net) to `./data/3d_models/models`.  

Each models(or projects) are seperated into directories. Naming convention is not a big problem unless you(and your code) can classify those.

**Since there must exist copyright issue, I will not provide the download script.**

Downloading the models took about 1~2 days, resulting with about 60000 projects.

Example:
```
(blender_py37) root@d0277a12bc8c:~/talking_head_anime_2# ls -al data/3d_models/models/ | head -7
total 4396
drwxr-xr-x 30157 31004 31000 1789952 Jan 28 18:31 .
drwxr-xr-x    10 31004 31000    4096 Feb 23 03:57 ..
drwxr-xr-x     3 31004 31000      60 Dec 24 00:20 3d.nicovideo__10002__ニュイ・ソシエール(部屋着)
drwxr-xr-x     3 31004 31000      58 Dec 24 00:20 3d.nicovideo__10003__こんにゃく式戌亥とこver1.0
drwxr-xr-x     3 31004 31000      73 Dec 24 00:20 3d.nicovideo__10024__(にじさんじ)ニュイ・ソシエールVer1.00
drwxr-xr-x     3 31004 31000      40 Dec 24 00:20 3d.nicovideo__10029__大阪ステージ
```

---

## Filtering Dataset (1)

For convinience, I used projects which used `.pmx`, `.pmd` or `.vrm` file.

After running the cells, I had about 30000 projects supporting `.pmd`, `.pmx`, or `.vrm` file.

In [1]:
import os
from tqdm import tqdm, trange


def find_model_in_dir(dir_model: str):
    """ searches if dir_model has loadable file

    Args:
        dir_model: root dir to find loadable file

    Returns:
        result: Boolean. True if exists, else False
        path_model: str. if result is true, valid model path is given. Else returns ''

    """
    result = False
    result_path = ''
    for root, subdirs, files in os.walk(dir_model):
        for file in files:
            path_model = os.path.join(root, file)
            if file.endswith('.pmx'):
                result = True
                result_path = path_model
                break
            elif file.endswith('.pmd'):
                result = True
                result_path = path_model
                break
            elif file.endswith('.vrm'):
                result = True
                result_path = path_model
                break

        if result:
            break

    return result, result_path


def find_valid_dirs(dir_root: str):
    """ finds subdir names which contains loadable files (.pmx, .pmd, .vrm)

    Args:
        dir_root: root dir to search
        path_save: path to save indices

    Returns:

    """
    models = sorted([os.path.join(dir_root, file) for file in os.listdir(dir_root)
                     if os.path.isdir(os.path.join(dir_root, file))])

    pmxs = []
    pmds = []
    vrms = []

    for dir_model in tqdm(models):
        result_bool, result_path = find_model_in_dir(dir_model)
        if result_bool:
            if result_path.endswith('.pmx'):
                pmxs.append(dir_model)
            elif result_path.endswith('.pmd'):
                pmds.append(dir_model)
            elif result_path.endswith('.vrm'):
                vrms.append(dir_model)

    print(len(models), len(pmxs), len(pmds), len(vrms))

    valid_list = sorted(pmxs + pmds + vrms)
    return valid_list


dir_root = './data/3d_models/models'
dir_projects = find_valid_dirs(dir_root)

100%|█████████████████████████████████████████████████████████████| 30155/30155 [00:02<00:00, 14299.80it/s]

30155 25715 3824 616


---

## Filtering dataset (2)

* First, non-human shaped models should be filtered out.
* Second, since original *Talking Head Anime* used 6 parameters(`Left eye`, `Right eye`, `Mouth`, `Head X`, `Head Y`, `Neck Z`), models not containing such keys should be filtered out.
    * Although `.pmx`, `.pmd`, `.vrm` is common and standard-like model and it is easy to figure out that using `あ`, `ウィンク`, `ウィンク右` and `頭`(or `頸`) is enough, but...
    * Some models use different keys. (e.g. some are translated to other language that is not `janapese`(standard)) 
    * To make sure, we'll extract shape keys and pose keys from all model files and analyze them.

### Finding all models from projects

Some projects include more than one models. (Usually cloth or accessories changed/added)

We find all `.pmd`, `.pmx`, `.vrm` files to enlarge training dataset.

In [2]:
def find_all_models(dir_projects):
    all_models = []
    for dir_project in tqdm(dir_projects):
        models_in_dir = []

        for root, subdirs, files in os.walk(dir_project):
            for file in files:
                path_model = os.path.abspath(os.path.join(root, file))
                extension = file.rsplit('.', 1)[-1].lower()
                if extension == 'pmd' or extension == 'pmx' or extension == 'vrm':
                    if path_model not in models_in_dir:
                        models_in_dir.append(path_model)
        
        all_models.extend(models_in_dir)

    return all_models

path_models = find_all_models(dir_projects[:20])
print(len(dir_projects), len(path_models))

with open('data/3d_models/all_models.txt', 'w', encoding='utf-8') as f:
    write_data = [item+'\n' for item in path_models]
    f.writelines(write_data)

100%|████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 1467.72it/s]

30155 38


### Saving models into .blend files

While extracting shape keys and pose keys, saving each model files into `.blend` file is helpful for later convenient use.

* `.blend` files can be loaded faster than loading from scratch with addons. 
* If once saved as `.blend` format, addons are not needed when loading & rendering `.blend` file.



**!!Before running!!** 
* Check `utils/data/run_blends.py` and `utils/data/save_to_blends.py` and try to adjust number of processes available with your device.( Line number ?? )

In [4]:
!python -m utils.data.run_blends

38it [03:07,  4.92s/it]


### Analysis of shape keys, pose keys

First, pair each key and models.

#### Find all .blend files

Using successfully saved `.blend` files helps to keep away from unexpected bugs.

In [5]:
# find all blend files

def find_all_blends(blend_dir):
    all_blends = []
    dirnames = os.listdir(blend_dir)
    for dirname in tqdm(dirnames):
        dir_model = os.path.join(blend_dir, dirname)
        blends_in_dir = []
        
        for root, subdirs, files in os.walk(dir_model):    
            for file in files:
                path_model = os.path.join(root, file)
                extension = file.rsplit('.', 1)[-1].lower()
                if extension == 'blend':
                    if path_model not in blends_in_dir:
                        blends_in_dir.append(path_model)
        
        all_blends.extend(blends_in_dir)

    return all_blends

all_blends = find_all_blends(blend_dir='data/3d_models/blends')
print(len(all_blends))

100%|████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 6394.73it/s]

38


In [6]:
from collections import defaultdict

def count_keys(path_blends):
    shapes = defaultdict(list)
    poses = defaultdict(list)
    
    for path_blend in tqdm(path_blends):
        path_shape = path_blend[:-6] + '.shape.txt'
        with open(path_shape, 'r', encoding='utf-8') as f:
            shape_keys = f.readlines()
        for line in shape_keys:
            key = line.strip()
            shapes[key].append(path_blend)
            
        path_pose = path_blend[:-6] + '.pose.txt'            
        with open(path_pose, 'r', encoding='utf-8') as f:
            pose_keys = f.readlines()
        for line in pose_keys:
            key = line.strip()
            poses[key].append(path_blend)
    
    return shapes, poses

shapes, poses = count_keys(all_blends)
shape_counts = [f'{key}, {len(value)}\n' for key, value in shapes.items()]
shape_counts = sorted(shape_counts, key=lambda x: int(x.strip().rsplit(', ')[-1]), reverse=True)
with open('data/3d_models/all_shapes.txt', 'w', encoding='utf-8') as f:
    f.writelines(shape_counts)


pose_counts = [f'{key}, {len(value)}\n' for key, value in poses.items()]
pose_counts = sorted(pose_counts, key=lambda x: int(x.strip().rsplit(', ')[-1]), reverse=True)
with open('data/3d_models/all_poses.txt', 'w', encoding='utf-8') as f:
    f.writelines(pose_counts)
    

100%|████████████████████████████████████████████████████████████████████| 38/38 [00:00<00:00, 3277.47it/s]


In [7]:
!cat data/3d_models/all_shapes.txt | head -10
print('########')
!cat data/3d_models/all_poses.txt | head -10

Basis, 38
まばたき, 20
い, 18
う, 18
え, 18
笑い, 18
怒り, 16
あ, 14
お, 14
なごみ, 14
########
全ての親, 25
センター, 25
上半身, 21
首, 21
頭, 21
肩.L, 21
腕.L, 21
肩.R, 21
腕.R, 21
下半身, 21


As expected, `あ`, `ウィンク`, `ウィンク右` and `頭` is the most general keys and using them would be enough.
* `あ` is much more used than `a` and amount of `a` could be ignored.

---

## Filtering dataset (3)

It is not guaranteed for a model to properly work with `あ`, `ウィンク`, `ウィンク右` and `頭`.

For example, somes models may move its' left eye with key(eye on the right when rendered at image) `ウィンク右`(meaning right wink), while others may move models' right eye.

Filtering and adjusting them is important proess in dataset generation. Sadly, such filtering could be best done with pure human effort. In this process, we render sample images from models and classify those.

#### Generating Sample Images

In [8]:
all_valid_blends = list(set.intersection(set(shapes['あ']), set(shapes['ウィンク']), set(shapes['ウィンク右']), set(poses['頭'])))
with open('data/3d_models/all_valid_blends.txt', 'w', encoding='utf-8') as f:
    write_lines = [item+'\n' for item in all_valid_blends]
    f.writelines(write_lines)
print(len(all_valid_blends))

10


In [37]:
!python -m utils.data.run_samples

10
10it [04:35, 27.50s/it]


#### Checking Generated Sample Images

**TODO: I'll try to make this runnable at ipynb.**

I used custom filtering helper tool to label the data.

You may use `python -m utils.data.filter_tool3` if you have gui in your device. Else, you should use your own method to check the samples.

1. run `python -m utils.data.filter_tool3` at the root of the project.
2. If the model is closing left eye at the second image, PRESS l.
3. If the model is closing right eye at the second image, PRESS r.
4. If there is some error on rendered images, PRESS any other key.



After checking sample images, you get `data/3d_models/filtered_idxs.txt`.

Now we will generate training dataset based on the labels.

## Generating Training Dataset

In [46]:
!python -m utils.data.generate_dataset

10
0it [00:00, ?it/s]ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4528:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4528:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4528:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5007:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2495:(snd_pcm_open_noupdate) Unknown PCM default
Traceback (most recent call last):
  File "/opt/conda/envs/blender_py37/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/conda/envs/blender_py37/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/root/talking_head_anime_2/uti